In [32]:
with open('../data/esempio.fa') as file: righe = file.readlines()

In [33]:
import re
geni = []
for riga in righe:
    riga = riga.rstrip('\n')
    if re.match('^>', riga):
        nuovo_gene = {'sequence': ''}
        geni.append(nuovo_gene)
        nuovo_gene['header'] = re.sub('^>', '', riga)
    elif not re.match('^;', riga):
        nuovo_gene['sequence'] = nuovo_gene['sequence'] + riga

Incollo da wikipedia.
Bisogna modificarla a mano, perchè lo start codon, M, W non hanno una forma compressa

In [34]:
tabella = """
Ala / A 	GCT, GCC, GCA, GCG 	GCN 	Leu / L 	TTA, TTG, CTT, CTC, CTA, CTG 	YTR, CTN
Arg / R 	CGT, CGC, CGA, CGG, AGA, AGG 	CGN, MGR 	Lys / K 	AAA, AAG 	AAR
Asn / N 	AAT, AAC 	AAY 	Met / M 	ATG 	ATG
Asp / D 	GAT, GAC 	GAY 	Phe / F 	TTT, TTC 	TTY
Cys / C 	TGT, TGC 	TGY 	Pro / P 	CCT, CCC, CCA, CCG 	CCN
Gln / Q 	CAA, CAG 	CAR 	Ser / S 	TCT, TCC, TCA, TCG, AGT, AGC 	TCN, AGY
Glu / E 	GAA, GAG 	GAR 	Thr / T 	ACT, ACC, ACA, ACG 	ACN
Gly / G 	GGT, GGC, GGA, GGG 	GGN 	Trp / W 	TGG 	TGG
His / H 	CAT, CAC 	CAY 	Tyr / Y 	TAT, TAC 	TAY
Ile / I 	ATT, ATC, ATA 	ATH 	Val / V 	GTT, GTC, GTA, GTG 	GTN
_START_ 	ATG	ATG 	_STOP_ 	TAA, TGA, TAG 	TAR, TRA
"""

In [35]:
righe = re.split("\n", tabella)

In [36]:
righe_divise = [re.split("\t", riga) for riga in righe if len(riga) > 1]

In [37]:
codice = {}
for riga in righe_divise:
    for codone in re.split(",", re.sub('\s', '', riga[1])):  
        codice[codone] = re.sub('(^.*\/|\s*)', '', riga[0])
    for codone in re.split(",", re.sub('\s', '', riga[4])):  
        codice[codone] = re.sub('(^.*\/|\s*)', '', riga[3])

In [38]:
def traduci(dna):
    proteina = ''
    while (len(dna) >= 3):
        codone = dna[0:3]
        proteina += codice[codone]
        dna = dna[3:]
    return proteina

def frame_strand(dna, frame, strand):
    if strand == '-':
            complemento = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A',}
            dna = "".join([complemento[carattere] for carattere in dna][::-1])
    return dna[frame:]

In [46]:
for gene in geni:
    for frame in [0, 1, 2]:
        for strand in ['+', '-']:
            tradotto = traduci(frame_strand(gene['sequence'], frame, strand)) 
            m = re.search('_START_.*_STOP_', tradotto)
            if m:
                gene['protein_' + str(frame) + strand] = m.group(0)

In [47]:
geni

[{'header': 'HSBGPG Human gene for bone gla protein (BGP)',
  'protein_0+': '_START_PLLIAGHSPEGINSAGGWRGRPAES_STOP_AAAQRSHRDT_START_RALTLLALLALAALCIAGQAGECPHLPSGRIAVGAERRKHHGPPLLTPLAGSPFAV_STOP_PPCCRLNPFAPALPLQRERREEQAARDAGEGG_STOP_GPWG_STOP_AGVNQAPFPLQVRSPAVQSPAKVQV_STOP_GWT_STOP_WVPGPSPLTLVPQSHSPTPATSCLAIRKASLLPT_STOP_SSQTQSHL_START_PAPLLHSLCVQAGGQRGSEETQALPVS_START_AGVREKAELGQGPASPGWSVGELQQGVASLGCGGGTGSLPWWAPWSP_START_CRERRDGHFARGL_START_PPRRVSQSPSPLPGSPGAQEGGV_STOP_AQSGL_STOP_RVG_STOP_PHRLSGGLSALLRPGLGCRSAGLAGNPSSAPLQAPFFPLPLALALTSQPYGCGVPIIPAAPK_STOP_',
  'protein_0-': '_START_D_STOP_ACNKVVRLQRDCQPKG_STOP_EEVGHGASSSQPPLQCGLRGGGGTHLPGQRCKVRPGPIGRGV_STOP_GLSWCLGGCAGLLLRTQLACPASLQHCLYPLGCAQR_STOP_',
  'protein_1+': '_START_AHLFSPLWLAVPLQSNHLVAGSIHLPQLCPCRGRGGKSKLPETQGKEDEGPGDELG_STOP_TRLPFLCRCEAQRCRVQQRCRYEDGPDGFLDPPLSPWSLSLIPPLLPPPVWPSGRPACSPPDPPKPRAT_STOP_CLPLCSTAFVSKQEGSEVVKRPRRYLYQWLG_STOP_EKRQSWAKALPLRDGLWGSCSREWPLWVVVGVQAACPGGHPGAPCVGRGG_START_GILHGG_STOP_CHHVGCLRAPVPYPDPLEPRRE

Considerare sequenze che si fermano al primo codone di stop

Considerare tutte le possibili coppie start/stop